In [46]:
import numpy as np
import matplotlib.pyplot as plt
import time
from subprocess import Popen, PIPE

In [47]:
def Bolsig(sigma_m,E_th,slope):
    sigma_m=sigma_m*1e-20
    slope=slope*1e-20
    f=open("LXCat0.txt","w")
    n=1000
    E=np.logspace(-3,1,n)

    f.write('----------------------\n')
    f.write('ELASTIC\nReid\n')
    f.write('1.371541e-4 / mass ratio\n')
    f.write('----------------------\n')
    
    for i in range(n):
        f.write(str(E[i])+' '+str(sigma_m)+'\n')

    f.write('----------------------\n')
    f.write('EXCITATION\nReid\n')
    f.write(str(E_th)+' / Thresold Energy\n')
    f.write('1.371541e-4 / mass ratio\n')
    f.write('----------------------\n')
    sigma_exc=slope*(E-E_th)*np.heaviside(E-E_th,1)
    for i in range(n):
        f.write(str(E[i])+' '+str(sigma_exc[i])+'\n')
    
    #plt.loglog(E,(sigma_m+sigma_exc)*1e20)
    #plt.show()

    f.close()
    
    t=time.time()
    process = Popen(['./bolsigminus', 'ex.dat'], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    #print (stdout)
    
    f=open("Reid.dat","r")
    n=128
    for line in f:
        if line[:8]=='E/N (Td)' and line[8:].strip()=='Mean energy (eV)':
            a=np.zeros(n)
            b=np.zeros(n)
            for i in range(n):
                l=f.readline().strip()
                if len(l)==0: break
                a[i], b[i] = [float(x) for x in l.split()]

    #print(a,b)
    #plt.loglog(a,b)
    p=np.polyfit(np.log10(a),np.log10(b),3)
    b_=10**np.polyval(p,np.log10(a))
    #plt.loglog(a,b_)
    err=np.sqrt(np.sum((b_-b)**2))
    print('time:',time.time()-t,'error:',err)
    #plt.show()
    return np.log10(b)

In [48]:
cross=open('Y_train.csv','w')
trans=open('X_train.csv','w')

n=100
sigma_m=np.random.rand(n)*9+1 #in (Angstrom)^2
E_th=np.random.rand(n)*0.9+0.1  # in eV 
slope=np.random.rand(n)*9+1 #in (Angstrom)^2
Y=np.zeros((n,3))
Y[:,0]=sigma_m
Y[:,1]=E_th
Y[:,2]=slope

np.savetxt(cross,Y,delimiter=',')
for i in range(n):
    X=Bolsig(Y[i,0],Y[i,1],Y[i,2])
    np.savetxt(trans,np.reshape(X,(1,-1)),delimiter=',')
    print(i,Y[i,:])
    #plt.plot(X)
    #plt.show()
cross.close()
trans.close()

time: 11.363693237304688 error: 0.26296157346718063
0 [8.57381125 0.89310382 2.10358964]
time: 10.101885318756104 error: 0.21526771077084622
1 [8.29801983 0.40045797 5.48799833]
time: 10.812005043029785 error: 0.3727343863191769
2 [4.02532164 0.47682346 2.81730141]
time: 11.190086126327515 error: 0.5522190934634323
3 [2.60877758 0.96746424 8.83596764]
time: 11.3759605884552 error: 0.299287722066759
4 [1.01036988 0.52492491 1.85417195]
time: 12.527436256408691 error: 0.36310969826857237
5 [1.39521199 0.56625213 4.05141396]
time: 25.116122722625732 error: 0.16815705511576015
6 [7.3614213  0.18519529 1.69887051]
time: 8.804679155349731 error: 0.19636380210822524
7 [8.61203036 0.36431686 8.91673185]
time: 10.480765581130981 error: 0.05878825103061705
8 [6.42495031 0.1121378  2.5198519 ]
time: 10.6239013671875 error: 0.3313192462782725
9 [6.35842627 0.85918833 5.25952493]
time: 10.272891521453857 error: 0.2928493026036838
10 [7.08186028 0.54579297 4.8608949 ]
time: 9.99803614616394 error: 0

time: 14.879054307937622 error: 0.25252994138487955
92 [8.32996255 0.66303608 9.28895199]
time: 14.90415334701538 error: 0.2496282645826854
93 [3.08836308 0.3125416  6.60045494]
time: 13.90109658241272 error: 0.3527519810327234
94 [5.37730465 0.66288735 6.33837818]
time: 12.354788780212402 error: 0.43548489042554334
95 [3.86502122 0.58675743 2.03197503]
time: 10.552233695983887 error: 0.25088534774776144
96 [6.15947722 0.38121025 5.71886354]
time: 13.047304630279541 error: 0.4909322575010902
97 [3.8872825  0.95021586 3.4788367 ]
time: 13.003556489944458 error: 0.2683753588141482
98 [5.35878557 0.39518862 8.03082735]
time: 14.252182006835938 error: 0.33458442251352744
99 [2.56796215 0.43816295 3.78503658]


In [49]:
cross=open('Y_test.csv','w')
trans=open('X_test.csv','w')

n=n//5
sigma_m=np.random.rand(n)*9+1 #in (Angstrom)^2
E_th=np.random.rand(n)*0.9+0.1  # in eV 
slope=np.random.rand(n)*9+1 #in (Angstrom)^2
Y=np.zeros((n,3))
Y[:,0]=sigma_m
Y[:,1]=E_th
Y[:,2]=slope

np.savetxt(cross,Y,delimiter=',')
for i in range(n):
    X=Bolsig(Y[i,0],Y[i,1],Y[i,2])
    np.savetxt(trans,np.reshape(X,(1,-1)),delimiter=',')
    print(i,Y[i,:])
cross.close()
trans.close()

time: 12.80091643333435 error: 0.26147758851989217
0 [9.72236286 0.20656399 1.65529716]
time: 12.630215644836426 error: 0.23618928610053838
1 [4.34685877 0.25801341 5.7849227 ]
time: 9.854990243911743 error: 0.21113807860227518
2 [9.75888481 0.44684883 4.84941014]
time: 10.712103843688965 error: 0.2682195703794591
3 [4.6142208  0.35701964 6.61970284]
time: 12.634669065475464 error: 0.39037462229444125
4 [4.55693736 0.75570429 9.35048487]
time: 12.644032716751099 error: 0.35307917069563555
5 [4.55795564 0.58019025 8.29536729]
time: 11.93283224105835 error: 0.03761271491121767
6 [8.15013536 0.10392934 3.78429787]
time: 13.99552321434021 error: 0.2130273323962979
7 [2.0348085  0.30290904 6.23593234]
time: 13.79978632926941 error: 0.20688247719105038
8 [4.31439308 0.22144365 6.93453965]
time: 12.94205379486084 error: 0.2731145586221026
9 [7.65854921 0.73958306 8.81772408]
time: 14.824334621429443 error: 0.35633681589651517
10 [5.108592   0.6964384  9.88774972]
time: 15.46114468574524 error